In [1]:
import sys
import gc
import wandb

import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import roc_auc_score
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizerFast

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
import os
os.environ["TOKENIZERS_PARALLELISM"] = "False"

In [2]:
import sklearn
import lightgbm

print(sklearn.__version__)
print(lightgbm.__version__)

## log into wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


1.2.2
3.3.2


wandb: Currently logged in as: peng_sun. Use `wandb login --relogin` to force relogin


True

In [3]:
test = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/test_essays.csv')
sub = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/sample_submission.csv')
org_train = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/train_essays.csv')

In [4]:
train = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/daigt/train_v2_drcat_02.csv', sep=',')
display(train.shape);

(44868, 5)

#### Read and append the generated data

In [5]:
train_gen_data = pd.read_parquet('/home/peng_sun2/s3shared/kaggle/llm-2023/external_data/gen_data_21122023.parquet')
display(train_gen_data.shape)

(18894, 5)

In [6]:
train = train.drop_duplicates(subset = ['text'])
train.reset_index(drop = True, inplace = True)

### append the generated data
# train = pd.concat([train, train_gen_data], axis = 0).reset_index(drop = True)
# display(train.shape)

In [7]:
train_0 = train.loc[train['label'] ==0, :].reset_index(drop = True)
train_1 = train.loc[train['label'] ==1, :].reset_index(drop = True)

In [8]:
train_0.shape

(27371, 5)

In [9]:
train_1.shape

(17497, 5)

In [10]:
# min_length = min(len(train_0), len(train_1))
# min_length

In [11]:
# sorted_train= pd.DataFrame()
# for i in tqdm(range(min_length), total = min_length):
#     sorted_train = pd.concat([sorted_train, pd.DataFrame(train_0.iloc[i,:]).T], axis = 0)
#     sorted_train = pd.concat([sorted_train, pd.DataFrame(train_1.iloc[i,:]).T], axis = 0)

In [12]:
# if len(train_0) > min_length:
#     sorted_train = pd.concat([sorted_train, train_0.iloc[min_length:, :]], axis = 0)
# elif len(train_1) > min_length:
#     sorted_train = pd.concat([sorted_train, train_1.iloc[min_length:, :]], axis = 0)

In [13]:
from tqdm import tqdm 
def sort_train(train):
    train_0 = train.loc[train['label'] ==0, :].reset_index(drop = True)
    train_1 = train.loc[train['label'] ==1, :].reset_index(drop = True)
    
    min_length = min(len(train_0), len(train_1))
    
    sorted_train= pd.DataFrame()
    for i in tqdm(range(min_length), total = min_length):
        sorted_train = pd.concat([sorted_train, pd.DataFrame(train_0.iloc[i,:]).T], axis = 0)
        sorted_train = pd.concat([sorted_train, pd.DataFrame(train_1.iloc[i,:]).T], axis = 0)
    
    if len(train_0) > min_length:
        sorted_train = pd.concat([sorted_train, train_0.iloc[min_length:, :]], axis = 0)
    elif len(train_1) > min_length:
        sorted_train = pd.concat([sorted_train, train_1.iloc[min_length:, :]], axis = 0)
        
    del train_0, train_1, train
    sorted_train['label'] = sorted_train['label'].astype(np.int32)
    return sorted_train.reset_index(drop = True)

In [14]:
display(train.shape)
train.head(10)

(44868, 5)

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False
5,Cell phone use should not be legal while drivi...,0,Phones and driving,persuade_corpus,False
6,Phones and Driving\n\nDriving is a good way to...,0,Phones and driving,persuade_corpus,False
7,PHONES AND DRIVING\n\nIn this world in which w...,0,Phones and driving,persuade_corpus,False
8,People are debating whether if drivers should ...,0,Phones and driving,persuade_corpus,False
9,Texting and driving\n\nOver half of drivers in...,0,Phones and driving,persuade_corpus,False


In [15]:
train = sort_train(train)

100%|██████████| 17497/17497 [00:49<00:00, 355.94it/s]


In [16]:
display(train.shape)
train.head(10)

(44868, 5)

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,"In recent years, technology has had a profoun...",1,Car-free cities,mistral7binstruct_v2,True
2,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
3,I strongly believe that meditation and mindful...,1,Distance learning,llama_70b_v1,False
4,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
5,One way school administrators can attempt to c...,1,Cell phones at school,chat_gpt_moth,False
6,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
7,While summer is meant as a break from the regu...,1,Summer projects,darragh_claude_v7,False
8,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False
9,The use of Facial Action Coding System (FACS) ...,1,Facial action coding system,darragh_claude_v6,True


In [17]:
LOWERCASE = False
VOCAB_SIZE = 30522

In [18]:
## Create byte-pair encoding tokenizer
raw_tokenizer = Tokenizer(models.BPE(unk_token='[UNK]'))

In [19]:
## adding normalization and pre_tokenizer
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()]+ [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

In [20]:
# Adding special tokens and creating trainer instance
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size = VOCAB_SIZE, special_tokens = special_tokens)

In [21]:
# creating huggingface dataset object
dataset = Dataset.from_pandas(test[['text']])
def train_corp_iter():
    for i in range(0, len(dataset), 1000):
        yield dataset[i:i+1000]['text']
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer = trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []
for text in tqdm(test['text'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = []

for text in tqdm(train['text'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))

/home/ubuntu/anaconda3/lib/python3.9/site-packages/pyarrow/pandas_compat.py:358: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


100%|██████████| 44868/44868 [02:14<00:00, 333.22it/s]


In [22]:
tokenized_texts_test[1]

['ĠBbb', 'Ġccc', 'Ġddd', '.']

In [23]:
def dummy(text):
    return text

In [24]:
vectorizer = TfidfVectorizer(ngram_range = (3, 5), lowercase=False, sublinear_tf = True, analyzer = 'word',
                            tokenizer = dummy,
                            preprocessor = dummy,
                            token_pattern = None,
                            strip_accents = 'unicode',
                            dtype = np.float32)
vectorizer.fit(tokenized_texts_test)

# Getting vocab
vocab = vectorizer.vocabulary_

print(vocab)

vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, 
                             #vocabulary=vocab,
                            analyzer = 'word',
                            tokenizer = dummy,
                            preprocessor = dummy,
                            token_pattern = None, strip_accents='unicode',
                            dtype = np.float32
                            )

tf_train = vectorizer.fit_transform(tokenized_texts_train)
tf_test = vectorizer.transform(tokenized_texts_test)
## get the vocabulary of the training dataset
train_vocab = vectorizer.vocabulary_

del vectorizer
gc.collect()

{'ĠAaa Ġbbb Ġccc': 0, 'Ġbbb Ġccc .': 6, 'ĠAaa Ġbbb Ġccc .': 1, 'ĠBbb Ġccc Ġddd': 2, 'Ġccc Ġddd .': 7, 'ĠBbb Ġccc Ġddd .': 3, 'ĠCCC Ġddd Ġeee': 4, 'Ġddd Ġeee .': 8, 'ĠCCC Ġddd Ġeee .': 5}


0

### Convert the type from numpy.float64 to numpy.float16

In [25]:
# import numpy as np
# from scipy.sparse import csr_matrix

# ## convert data for tf_train
# data  = tf_train.data.astype(np.float16)
# indices = tf_train.indices
# indptr = tf_train.indptr
# tf_train = csr_matrix((data, indices, indptr), shape = tf_train.shape)


# ## convert data for tf_test
# data = tf_test.data.astype(np.float16)
# indices = tf_test.indices
# indptr = tf_test.indptr
# tf_test = csr_matrix((data, indices, indptr), shape = tf_test.shape)

# del data, indices, indptr
# gc.collect()

## Start training

In [26]:
# ### save train_vocab
# import joblib
# model_path = '/home/peng_sun2/s3shared/kaggle/llm-2023/baseline1/'
# joblib.dump(train_vocab, f'{model_path}vocab_train.pkl')

In [27]:


# # Check if it's in scoring stage
# if len(test.text.values) <= 5:
#     # if not, just sample submission
#     sub.to_csv('submission.csv', index=False)
# else:
# otherwise, run fitting process

sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'validatoin_auc', 'goal': 'maximize'},
    'parameters': {
        'iterations': {'min': 500, 'max': 1500},
        'learning_rate': {'min': 0.005, 'max': 0.3},
        'depth': {'min': 4, 'max': 10},
        'l2_leaf_reg': {'min': 0.5, 'max': 1.0},
        'bagging_temperature': {'min': 0.0, 'max': 1.0},
        'border_count': {'min': 32, 'max': 255}
    }
}

sweep_id = wandb.sweep(sweep_config, project="llm-2023-catboost-hyperparameter")


Create sweep with ID: n9tseh37
Sweep URL: https://wandb.ai/peng_sun/llm-2023-catboost-hyperparameter/sweeps/n9tseh37


In [28]:
def split_range_into_parts(start, end, num_parts):
    splits = np.linspace(start, end, num_parts + 1)
    ranges = [(int(np.floor(splits[i])), int(np.ceil(splits[i + 1]))) for i in range(num_parts)]
    return np.array(ranges)

In [29]:
def increment_train_catboost(tf_train, y_train, cat, num_parts):
    split_ranges = split_range_into_parts(0, tf_train.shape[0], num_parts);
    #print(split_ranges)
    
    ## incrementally train catboost
    for i, (start, end) in enumerate(split_ranges):
        print('start, end ', start, ' ', end)
        train_X_split = tf_train[start:end, :]
        train_y_split = y_train[start:end]
        
        #print(train_X_split.shape, train_y_split.shape)
        if i ==0:
            cat.fit(train_X_split, train_y_split)
        else: 
            cat.fit(train_X_split, train_y_split, init_model = cat)
        print(f'part {i} is finished ...')
    return cat
    

In [30]:
y_train = train['label'].values

In [31]:
from sklearn.model_selection import StratifiedGroupKFold
from catboost import CatBoostClassifier
from sklearn import metrics
import lightgbm as lgb
## cv split
skf = StratifiedGroupKFold(n_splits = 5)
skf.get_n_splits(tf_train, y_train, groups =train['prompt_name'])

def train_sweep():
    with wandb.init() as run:
        config = wandb.config
        for i, (train_idx, test_idx) in enumerate(skf.split(tf_train, y_train, groups = train['prompt_name'])):
            train_X = tf_train[train_idx]
            eval_X = tf_train[test_idx];
            train_y = y_train[train_idx]
            eval_y = y_train[test_idx];
        
            
            params = {
                'iterations': config.iterations,
                'learning_rate': config.learning_rate,
                'depth': config.depth,
                'l2_leaf_reg': config.l2_leaf_reg,
                'bagging_temperature': config.bagging_temperature,
                'border_count': config.border_count
            }
            
            cat=CatBoostClassifier(
                **params,
                allow_const_label=True,
                #used_ram_limit="2048MB"
            )
            cat = increment_train_catboost(train_X, train_y, cat, num_parts = 2)
            cat.fit(train_X, train_y)
            
            eval_preds = cat.predict_proba(eval_X)[:, 1]
            fpr, tpr, thresholds = metrics.roc_curve(eval_y, eval_preds, pos_label = 1)
            eval_auc = metrics.auc(fpr, tpr)
            wandb.log({
                'validatoin_auc': eval_auc
            });
wandb.agent(sweep_id, function = train_sweep)

wandb: Agent Starting Run: dxsc25hv with config:
wandb: 	bagging_temperature: 0.4387082078072594
wandb: 	border_count: 191
wandb: 	depth: 6
wandb: 	iterations: 903
wandb: 	l2_leaf_reg: 0.9047130924242102
wandb: 	learning_rate: 0.059811165624044225
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


start, end  0   18009
0:	learn: 0.6183396	total: 515ms	remaining: 7m 44s
1:	learn: 0.5549061	total: 873ms	remaining: 6m 33s
2:	learn: 0.5068323	total: 1.26s	remaining: 6m 17s
3:	learn: 0.4613806	total: 1.63s	remaining: 6m 5s
4:	learn: 0.4270479	total: 1.99s	remaining: 5m 57s
5:	learn: 0.4008428	total: 2.34s	remaining: 5m 50s
6:	learn: 0.3738864	total: 2.71s	remaining: 5m 46s
7:	learn: 0.3531735	total: 3.08s	remaining: 5m 44s
8:	learn: 0.3339836	total: 3.46s	remaining: 5m 43s
9:	learn: 0.3205993	total: 3.83s	remaining: 5m 42s
10:	learn: 0.3069338	total: 4.2s	remaining: 5m 40s
11:	learn: 0.2936984	total: 4.57s	remaining: 5m 39s
12:	learn: 0.2825544	total: 4.95s	remaining: 5m 38s
13:	learn: 0.2718257	total: 5.33s	remaining: 5m 38s
14:	learn: 0.2629642	total: 5.7s	remaining: 5m 37s
15:	learn: 0.2535763	total: 6.07s	remaining: 5m 36s
16:	learn: 0.2443358	total: 6.43s	remaining: 5m 35s
17:	learn: 0.2367779	total: 6.79s	remaining: 5m 33s
18:	learn: 0.2306610	total: 7.14s	remaining: 5m 32s
19:

158:	learn: 0.0473604	total: 56.5s	remaining: 4m 24s
159:	learn: 0.0472576	total: 56.8s	remaining: 4m 23s
160:	learn: 0.0469765	total: 57.1s	remaining: 4m 23s
161:	learn: 0.0466606	total: 57.5s	remaining: 4m 22s
162:	learn: 0.0462775	total: 57.8s	remaining: 4m 22s
163:	learn: 0.0461620	total: 58.1s	remaining: 4m 21s
164:	learn: 0.0460419	total: 58.5s	remaining: 4m 21s
165:	learn: 0.0458483	total: 58.8s	remaining: 4m 21s
166:	learn: 0.0456813	total: 59.1s	remaining: 4m 20s
167:	learn: 0.0456396	total: 59.5s	remaining: 4m 20s
168:	learn: 0.0453593	total: 59.8s	remaining: 4m 19s
169:	learn: 0.0452092	total: 1m	remaining: 4m 19s
170:	learn: 0.0450696	total: 1m	remaining: 4m 18s
171:	learn: 0.0448432	total: 1m	remaining: 4m 18s
172:	learn: 0.0445860	total: 1m 1s	remaining: 4m 18s
173:	learn: 0.0442230	total: 1m 1s	remaining: 4m 17s
174:	learn: 0.0439353	total: 1m 1s	remaining: 4m 17s
175:	learn: 0.0438601	total: 1m 2s	remaining: 4m 16s
176:	learn: 0.0435977	total: 1m 2s	remaining: 4m 16s
17

312:	learn: 0.0237999	total: 1m 49s	remaining: 3m 26s
313:	learn: 0.0236705	total: 1m 50s	remaining: 3m 26s
314:	learn: 0.0235857	total: 1m 50s	remaining: 3m 26s
315:	learn: 0.0234796	total: 1m 50s	remaining: 3m 25s
316:	learn: 0.0234674	total: 1m 51s	remaining: 3m 25s
317:	learn: 0.0233582	total: 1m 51s	remaining: 3m 25s
318:	learn: 0.0232586	total: 1m 51s	remaining: 3m 24s
319:	learn: 0.0232461	total: 1m 52s	remaining: 3m 24s
320:	learn: 0.0232073	total: 1m 52s	remaining: 3m 23s
321:	learn: 0.0230413	total: 1m 52s	remaining: 3m 23s
322:	learn: 0.0229386	total: 1m 53s	remaining: 3m 23s
323:	learn: 0.0229286	total: 1m 53s	remaining: 3m 22s
324:	learn: 0.0228500	total: 1m 53s	remaining: 3m 22s
325:	learn: 0.0227688	total: 1m 54s	remaining: 3m 21s
326:	learn: 0.0227129	total: 1m 54s	remaining: 3m 21s
327:	learn: 0.0227026	total: 1m 54s	remaining: 3m 21s
328:	learn: 0.0225598	total: 1m 55s	remaining: 3m 20s
329:	learn: 0.0225498	total: 1m 55s	remaining: 3m 20s
330:	learn: 0.0224904	total:

466:	learn: 0.0138064	total: 2m 41s	remaining: 2m 30s
467:	learn: 0.0137652	total: 2m 41s	remaining: 2m 30s
468:	learn: 0.0137204	total: 2m 42s	remaining: 2m 30s
469:	learn: 0.0136581	total: 2m 42s	remaining: 2m 29s
470:	learn: 0.0136253	total: 2m 43s	remaining: 2m 29s
471:	learn: 0.0135906	total: 2m 43s	remaining: 2m 29s
472:	learn: 0.0135334	total: 2m 43s	remaining: 2m 28s
473:	learn: 0.0135288	total: 2m 44s	remaining: 2m 28s
474:	learn: 0.0134834	total: 2m 44s	remaining: 2m 28s
475:	learn: 0.0134789	total: 2m 44s	remaining: 2m 27s
476:	learn: 0.0134488	total: 2m 45s	remaining: 2m 27s
477:	learn: 0.0134048	total: 2m 45s	remaining: 2m 27s
478:	learn: 0.0133636	total: 2m 45s	remaining: 2m 26s
479:	learn: 0.0133189	total: 2m 46s	remaining: 2m 26s
480:	learn: 0.0132792	total: 2m 46s	remaining: 2m 25s
481:	learn: 0.0132751	total: 2m 46s	remaining: 2m 25s
482:	learn: 0.0132207	total: 2m 47s	remaining: 2m 25s
483:	learn: 0.0131737	total: 2m 47s	remaining: 2m 24s
484:	learn: 0.0131696	total:

...

717:	learn: 0.0012735	total: 10m 48s	remaining: 7m 26s
718:	learn: 0.0012735	total: 10m 49s	remaining: 7m 25s
719:	learn: 0.0012735	total: 10m 50s	remaining: 7m 24s
720:	learn: 0.0012683	total: 10m 51s	remaining: 7m 23s
721:	learn: 0.0012683	total: 10m 52s	remaining: 7m 22s
722:	learn: 0.0012683	total: 10m 53s	remaining: 7m 21s
723:	learn: 0.0012683	total: 10m 54s	remaining: 7m 20s
724:	learn: 0.0012678	total: 10m 55s	remaining: 7m 20s
725:	learn: 0.0012678	total: 10m 55s	remaining: 7m 19s
726:	learn: 0.0012597	total: 10m 56s	remaining: 7m 18s
727:	learn: 0.0012597	total: 10m 57s	remaining: 7m 17s
728:	learn: 0.0012596	total: 10m 58s	remaining: 7m 16s
729:	learn: 0.0012591	total: 10m 59s	remaining: 7m 15s
730:	learn: 0.0012591	total: 11m	remaining: 7m 14s
731:	learn: 0.0012591	total: 11m 1s	remaining: 7m 13s
732:	learn: 0.0012591	total: 11m 2s	remaining: 7m 12s
733:	learn: 0.0012591	total: 11m 3s	remaining: 7m 11s
734:	learn: 0.0012591	total: 11m 3s	remaining: 7m 10s
735:	learn: 0.0012

868:	learn: 0.0011402	total: 13m 4s	remaining: 5m 9s
869:	learn: 0.0011401	total: 13m 5s	remaining: 5m 8s
870:	learn: 0.0011401	total: 13m 6s	remaining: 5m 7s
871:	learn: 0.0011401	total: 13m 6s	remaining: 5m 6s
872:	learn: 0.0011401	total: 13m 7s	remaining: 5m 5s
873:	learn: 0.0011401	total: 13m 8s	remaining: 5m 5s
874:	learn: 0.0011401	total: 13m 9s	remaining: 5m 4s
875:	learn: 0.0011401	total: 13m 10s	remaining: 5m 3s
876:	learn: 0.0011401	total: 13m 11s	remaining: 5m 2s
877:	learn: 0.0011401	total: 13m 12s	remaining: 5m 1s
878:	learn: 0.0011401	total: 13m 13s	remaining: 5m
879:	learn: 0.0011400	total: 13m 14s	remaining: 4m 59s
880:	learn: 0.0011401	total: 13m 14s	remaining: 4m 58s
881:	learn: 0.0011400	total: 13m 15s	remaining: 4m 57s
882:	learn: 0.0011400	total: 13m 16s	remaining: 4m 56s
883:	learn: 0.0011401	total: 13m 17s	remaining: 4m 55s
884:	learn: 0.0011400	total: 13m 18s	remaining: 4m 55s
885:	learn: 0.0011399	total: 13m 19s	remaining: 4m 54s
886:	learn: 0.0011399	total: 13

1019:	learn: 0.0010741	total: 15m 19s	remaining: 2m 53s
1020:	learn: 0.0010740	total: 15m 20s	remaining: 2m 52s
1021:	learn: 0.0010663	total: 15m 21s	remaining: 2m 51s
1022:	learn: 0.0010663	total: 15m 22s	remaining: 2m 50s
1023:	learn: 0.0010663	total: 15m 23s	remaining: 2m 49s
1024:	learn: 0.0010663	total: 15m 24s	remaining: 2m 48s
1025:	learn: 0.0010663	total: 15m 24s	remaining: 2m 47s
1026:	learn: 0.0010663	total: 15m 25s	remaining: 2m 46s
1027:	learn: 0.0010663	total: 15m 26s	remaining: 2m 45s
1028:	learn: 0.0010663	total: 15m 27s	remaining: 2m 44s
1029:	learn: 0.0010663	total: 15m 28s	remaining: 2m 44s
1030:	learn: 0.0010663	total: 15m 29s	remaining: 2m 43s
1031:	learn: 0.0010663	total: 15m 30s	remaining: 2m 42s
1032:	learn: 0.0010663	total: 15m 31s	remaining: 2m 41s
1033:	learn: 0.0010663	total: 15m 31s	remaining: 2m 40s
1034:	learn: 0.0010662	total: 15m 32s	remaining: 2m 39s
1035:	learn: 0.0010662	total: 15m 33s	remaining: 2m 38s
1036:	learn: 0.0010662	total: 15m 34s	remaining:

1167:	learn: 0.0010204	total: 17m 31s	remaining: 39.6s
1168:	learn: 0.0010205	total: 17m 32s	remaining: 38.7s
1169:	learn: 0.0010204	total: 17m 33s	remaining: 37.8s
1170:	learn: 0.0010204	total: 17m 34s	remaining: 36.9s
1171:	learn: 0.0010205	total: 17m 34s	remaining: 36s
1172:	learn: 0.0010205	total: 17m 35s	remaining: 35.1s
1173:	learn: 0.0010204	total: 17m 36s	remaining: 34.2s
1174:	learn: 0.0010205	total: 17m 37s	remaining: 33.3s
1175:	learn: 0.0010204	total: 17m 38s	remaining: 32.4s
1176:	learn: 0.0010204	total: 17m 39s	remaining: 31.5s
1177:	learn: 0.0010204	total: 17m 40s	remaining: 30.6s
1178:	learn: 0.0010204	total: 17m 41s	remaining: 29.7s
1179:	learn: 0.0010204	total: 17m 41s	remaining: 28.8s
1180:	learn: 0.0010204	total: 17m 42s	remaining: 27.9s
1181:	learn: 0.0010204	total: 17m 43s	remaining: 27s
1182:	learn: 0.0010204	total: 17m 44s	remaining: 26.1s
1183:	learn: 0.0010204	total: 17m 45s	remaining: 25.2s
1184:	learn: 0.0010204	total: 17m 46s	remaining: 24.3s
1185:	learn: 0

wandb: Ctrl + C detected. Stopping sweep.


In [32]:
wandb.finish()

82:	learn: 0.0375781	total: 1m 37s	remaining: 22m 6s
83:	learn: 0.0374514	total: 1m 38s	remaining: 22m 4s
84:	learn: 0.0369669	total: 1m 39s	remaining: 22m 2s
85:	learn: 0.0365505	total: 1m 40s	remaining: 22m
86:	learn: 0.0362368	total: 1m 42s	remaining: 21m 59s


validatoin_auc,█▁
validatoin_auc,0.9951


87:	learn: 0.0359062	total: 1m 43s	remaining: 21m 59s
88:	learn: 0.0356427	total: 1m 44s	remaining: 21m 59s
89:	learn: 0.0355391	total: 1m 45s	remaining: 21m 58s
90:	learn: 0.0350992	total: 1m 46s	remaining: 21m 57s
91:	learn: 0.0348529	total: 1m 48s	remaining: 21m 57s
92:	learn: 0.0344653	total: 1m 49s	remaining: 21m 57s
93:	learn: 0.0342141	total: 1m 50s	remaining: 21m 56s
94:	learn: 0.0341602	total: 1m 51s	remaining: 21m 55s
95:	learn: 0.0338880	total: 1m 53s	remaining: 21m 54s
96:	learn: 0.0334956	total: 1m 54s	remaining: 21m 54s
97:	learn: 0.0332422	total: 1m 55s	remaining: 21m 52s
98:	learn: 0.0328433	total: 1m 56s	remaining: 21m 51s
99:	learn: 0.0326930	total: 1m 57s	remaining: 21m 49s
100:	learn: 0.0321720	total: 1m 58s	remaining: 21m 48s
101:	learn: 0.0318552	total: 2m	remaining: 21m 46s
102:	learn: 0.0317629	total: 2m 1s	remaining: 21m 44s
103:	learn: 0.0311498	total: 2m 2s	remaining: 21m 44s
104:	learn: 0.0308623	total: 2m 3s	remaining: 21m 42s
105:	learn: 0.0305612	total: 2

### Best parameters

* colsample_bynode = 0.9669
* colsample_bytree = 0.9645
* lambda_l1 = 0.7217
* lambda_l2 = 3.207
* learning_rate = 0.09574
* max_bin = 184
* max_depth = 62
* min_data_in_leaf = 111
